# Generate samples from previously chosen verbs, adjectives and identities

The goal of this notebook is to create a synthetic dataset that can be used to measure the unintended bias in a toxicity classifier. 

### Prepare functions, data etc.

In [1]:
# import libraries
import re
import pandas as pd
from typing import List

In [2]:
# utility functions
def read_content(filename:str) -> List[str]:
    """
    Opens file and returns its contents as a list split by newlines.
    """
    f = open(filename)
    contents = f.read().split("\n")
    f.close()
    return contents

def get_word_pos(wordtype:str, wordlist:List[str]) -> int:
    """
    Returns the position of the word to be inserted in list of words.
    """
    for i, word in enumerate(wordlist):
        if wordtype.upper() == "VERB":
            if "VERB" in word:
                return i
        elif wordtype.upper() == "IDENTITY":
            if "IDENTITY" in word:
                return i
        elif wordtype.upper() == "ARTICLE":
            if "ARTICLE" in word:
                return i
        elif wordtype.upper() == "ADJECTIVE":
            if "ADJECTIVE" in word:
                return i
        else:
            return None

def determine_verb_list(verbtype:str, polarity:str) -> List[str]:
    """
    Determines which verb list to use based on the verbtype specified in the sample
    and returns its contents.
    """
    if polarity == "neg":
        if verbtype == "VERB_PRS":
            return read_content("verbs/neg_prs.txt") # present tense
        elif verbtype == "VERB_IMP":
            return read_content("verbs/neg_imp.txt") # imperative
        elif verbtype == "VERB_PASS":
            return read_content("verbs/neg_pass.txt") # passive
    elif polarity == "pos":
        if verbtype == "VERB_PRS":
            return read_content("verbs/pos_prs.txt") # present tense
        elif verbtype == "VERB_IMP":
            return read_content("verbs/pos_imp.txt") # imperative
        elif verbtype == "VERB_PASS":
            return read_content("verbs/pos_pass.txt") # passive
    return None
    
def determine_identity_list(identitytype:str):
    """
    Determines which identity list to use based on the type specified in the sample
    and returns its contents.
    """
    identitytype = re.sub(r'[^A-z]','', identitytype) # remove punctuation from identity type
    if identitytype == "IDENTITY_PL":
        return read_content("identities/pl.txt")
    elif identitytype == "IDENTITY_SG":
        return read_content("identities/sg.txt")
    else:
        return None

In [3]:
# main function
def generate_samples(templates:List[str], polarity:str=None) -> List[str]:
    """
    Generate samples from templates by slotting in identities and if necessary, verbs and adjectives. Returns a list of the generated samples.
    
    Args:
        templates (List[str]): list of templates as strings.
        polarity (str): allows the values "pos" and "neg" that specifies whether the sentiment of the verbs/adjectives is positive or negative. 

    Returns:
        List[str]: list of generated samples as strings.
    """
    
    generated_samples = []

    # go through templates
    for template in templates:

        sample = template.split()
        
        # get identity position and list
        IDENTITY_POS = get_word_pos("identity", sample)
        IDENTITY_LIST = determine_identity_list(sample[IDENTITY_POS])
        IS_PLURAL = sample[IDENTITY_POS].endswith("_PL") # check for plurality

        # get verb position and list (if relevant)
        VERB_POS = get_word_pos("verb", sample)
        if VERB_POS is not None: # i.e. if there's a verb slot in the template
            VERB_LIST = determine_verb_list(sample[VERB_POS], polarity)
        
        # get article position
        ART_POS = get_word_pos("article", sample)
        
        # get adjective position
        ADJ_POS = get_word_pos("adjective", sample)

        # create one version of this template per identity
        for identity in IDENTITY_LIST:
            
            # if the article has its own position in the sentence, move it here
            # otherwise it is kept as part of the NP that the identities consist of
            if ART_POS is not None:            
                # split the identity phrase into article and the rest
                words = identity.split()
                article = words[0]
                identity = ' '.join(words[1:])

                # insert article at article position
                sample[ART_POS] = article
            else:
                article = None
            
            # insert noun at IDENTITY position
            sample[IDENTITY_POS] = identity
            
            # then create one version of this per verb, if necessary
            if VERB_POS is not None:
                for verb in VERB_LIST:
                    sample[VERB_POS] = verb
                    generated_samples.append(' '.join(sample)) # SAMPLE WITH VERB
            
            # then create one version of this per adjective, if necessary
            elif ADJ_POS is not None:
                for adj in read_content(f"adjectives/{polarity}.txt"):
                    conjugations = adj.split(",") # conjugations in the order: standard, neuter, plural
            
                    # if neuter article or noun, use neuter conjugation
                    if article == "et" or sample[0].lower() == "det": 
                        adj = conjugations[1].strip()
            
                    # if plural noun phrase, use plural adjective
                    elif IS_PLURAL:
                        adj = conjugations[2].strip()
                    
                    # else use standard version            
                    else:
                        adj = conjugations[0].strip()
                        
                    # insert adjective at ADJ position and add sample to list
                    sample[ADJ_POS] = adj 
                    generated_samples.append(' '.join(sample)) # SAMPLE WITH ADJECTIVE
                
            else: # i.e. if no verb or adjective is needed, create sample
                generated_samples.append(' '.join(sample)) # SAMPLE WITHOUT VERB OR ADJECTIVE
                
    print(len(generated_samples), "samples created")
    
    return generated_samples

In [4]:
print(len(read_content("verbs/neg_imp.txt"))*2, "verbs")
print(len(read_content("identities/sg.txt")), "identities")
print(len(read_content("adjectives/pos.txt"))*2, "adjectives")

26 verbs
45 identities
50 adjectives


### Create samples for templates that are ALWAYS toxic such as "I hate all [identity]"

In [5]:
# open templates that are always toxic
templates = read_content("templates/always_neg.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45):,} samples (3 templates * 45 identities)")

3 templates:
['IDENTITY_PL er klamme og generelt bare nedern', 'Jeg er IDENTITY_SG, og jeg hader dig og synes, du er træls', 'Jeg hader alle IDENTITY_PL']

Should get 135 samples (3 templates * 45 identities)


In [6]:
# generate samples using these templates

generated_samples = generate_samples(templates)
df1 = pd.DataFrame(generated_samples, columns=['text'])
df1['toxic'] = 1 # these sentences are always non-toxic/neutral
df1.head()

135 samples created


,text,toxic
0,kvinder er klamme og generelt bare nedern,1
1,piger er klamme og generelt bare nedern,1
2,damer er klamme og generelt bare nedern,1
3,koner er klamme og generelt bare nedern,1
4,kvindfolk er klamme og generelt bare nedern,1


### Create samples for templates that are ALWAYS non-toxic such as "I am a/an [identity]"

In [7]:
# open templates that are always non-toxic/neutral
templates = read_content("templates/always_neu.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45):,} samples (3 templates * 45 identities)")

3 templates:
['Jeg er IDENTITY_SG, spørg mig om alt', 'IDENTITY_PL er ligesom alle andre', 'Jeg er IDENTITY_SG']

Should get 135 samples (3 templates * 45 identities)


In [8]:
# generate samples using these templates

generated_samples = generate_samples(templates)
df2 = pd.DataFrame(generated_samples, columns=['text'])
df2['toxic'] = 0 # these sentences are always non-toxic/neutral
df2.head()

135 samples created


,text,toxic
0,Jeg er en kvinde spørg mig om alt,0
1,Jeg er en pige spørg mig om alt,0
2,Jeg er en dame spørg mig om alt,0
3,Jeg er en kone spørg mig om alt,0
4,Jeg er et kvindfolk spørg mig om alt,0


### Create samples for templates whose toxicity depend on the polarity of the verb such as "I [verb] [identities]"

In [9]:
# open templates that depend on the verb
templates = read_content("templates/verb_dependent.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(3*45*26):,} samples (3 templates * 45 identities * 26 verbs)")
print("Half should be negative/toxic, half positive/non-toxic")

3 templates:
['VERB_IMP IDENTITY_PL', 'Jeg VERB_PRS IDENTITY_PL', 'IDENTITY_PL bør VERB_PASS']

Should get 3,510 samples (3 templates * 45 identities * 26 verbs)
Half should be negative/toxic, half positive/non-toxic


In [10]:
# generate samples using these templates

# toxic/negative samples
generated_samples = generate_samples(templates, "neg")
df3 = pd.DataFrame(generated_samples, columns=['text'])
df3['toxic'] = 1 
df3.head()

1755 samples created


,text,toxic
0,afsky kvinder,1
1,dræb kvinder,1
2,eliminer kvinder,1
3,had kvinder,1
4,henret kvinder,1


In [11]:
# non-toxic/positive samples
generated_samples = generate_samples(templates, "pos")
df4 = pd.DataFrame(generated_samples, columns=['text'])
df4['toxic'] = 0 
df4.head()

1755 samples created


,text,toxic
0,beundr kvinder,0
1,eftertragt kvinder,0
2,elsk kvinder,0
3,fejr kvinder,0
4,hædr kvinder,0


### Create samples for templates whose toxicity depend on the polarity of the adjective such as "[identities] are [adjective]"

In [12]:
# open templates that depend on the adjective
templates = read_content("templates/adjective_dependent.txt")
print(len(templates), "templates:")
print(templates)
print(f"\nShould get {(4*45*50):,} samples (4 templates * 45 identities * 50 adjectives)")
print("Half should be negative/toxic, half positive/non-toxic")

4 templates:
['ADJECTIVE IDENTITY_PL', 'Du er ARTICLE ADJECTIVE IDENTITY_SG', 'Det er ADJECTIVE at være IDENTITY_SG', 'IDENTITY_PL er ADJECTIVE']

Should get 9,000 samples (4 templates * 45 identities * 50 adjectives)
Half should be negative/toxic, half positive/non-toxic


In [13]:
# generate samples using these templates

# toxic/negative samples
generated_samples = generate_samples(templates, "neg")
df5 = pd.DataFrame(generated_samples, columns=['text'])
df5['toxic'] = 1 
df5.head()

4500 samples created


,text,toxic
0,dårlige kvinder,1
1,gale kvinder,1
2,kriminelle kvinder,1
3,onde kvinder,1
4,lede kvinder,1


In [14]:
# non-toxic/positive samples
generated_samples = generate_samples(templates, "pos")
df6 = pd.DataFrame(generated_samples, columns=['text'])
df6['toxic'] = 0 
df6.head()

4500 samples created


,text,toxic
0,sjove kvinder,0
1,glade kvinder,0
2,super kvinder,0
3,fantastiske kvinder,0
4,spændende kvinder,0


### Combine all temporary dataframes into a single one and save as xlsx-file

In [15]:
print("Number of samples in each df:\n")
print("Always toxic (no verb/adj)    : ", len(df1))
print("Always non-toxic (no verb/adj): ", len(df2))
print("Toxic because of verb         : ", len(df3))
print("Non-toxic because of verb     : ", len(df4))
print("Toxic because of adjective    : ", len(df5))
print("Non-toxic because of adjective: ", len(df6))
print()
print(f"Total                         = {sum([len(df1), len(df2), len(df3), len(df4), len(df5), len(df6)]):,} samples")

Number of samples in each df:

Always toxic (no verb/adj)    :  135
Always non-toxic (no verb/adj):  135
Toxic because of verb         :  1755
Non-toxic because of verb     :  1755
Toxic because of adjective    :  4500
Non-toxic because of adjective:  4500

Total                         = 12,780 samples


In [16]:
synthetic_data = pd.concat([df1, df2, df3, df4, df5, df6])
synthetic_data.drop_duplicates() # ensure that theres no duplicates
synthetic_data.to_excel("synthetic_data.xlsx") 
synthetic_data

,text,toxic
0,kvinder er klamme og generelt bare nedern,1
1,piger er klamme og generelt bare nedern,1
2,damer er klamme og generelt bare nedern,1
3,koner er klamme og generelt bare nedern,1
4,kvindfolk er klamme og generelt bare nedern,1
...,...,...
4495,genderqueer personer er suveræne,0
4496,genderqueer personer er herlige,0
4497,genderqueer personer er fornemme,0
4498,genderqueer personer er ekstraordinære,0


In [17]:
# how many samples are toxic and not?
print(synthetic_data['toxic'].value_counts())

toxic
1    6390
0    6390
Name: count, dtype: int64
